# MapReduce

## PageRank using Map-Reduce

The task involves implementing the PageRank algorithm using the MapReduce paradigm in Python with the MRJob library. PageRank is an algorithm used by search engines to rank web pages in their search results. It assigns a numerical weight to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of measuring its relative importance within the set.

The challenge here is to simulate the MapReduce process for PageRank, even without access to a Hadoop cluster. I'll be focusing on understanding the mindset of MapReduce and implementing the algorithm in a smaller scale using Python and MRJob.

In [25]:
%%writefile mrpagerank.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import argparse
import shutil

# node_info = [PageRank, num_outlinks, outlinks] 
N = 0
D = 0

# padgett_proc.txt
class MRPageRank(MRJob):
    def configure_args(self):
        super(MRPageRank, self).configure_args()
        self.add_passthru_arg('--iter', type=int, default=5, help='Number of iterations')
        self.add_passthru_arg('-s', type=float, default=0.85, help='Damping factor')

    def ip_mapper(self, _, line):
        node_id, node_info = line.split('\t')
        pr, outlinks = eval(node_info)

        if len(outlinks) == 0:
            yield 1, [pr, line]
        else:
            yield 1, [0, line]
    
    def ip_reducer(self, key, values):
        global N, D
        ips = []
        lines = []
        N = 0
        for value in values:
            ips.append(value[0])
            lines.append(value[1])
            N += 1
        
        D = sum(ips)

        for line in lines:
            yield None, line
    
    def pr_mapper(self, _, line):
        node_id, node_info = line.split('\t')
        node_id = int(node_id)
        pr, outlinks = eval(node_info)
        needed_infos = [node_id, outlinks]

        yield node_id, [0.0, needed_infos]
        for outlink in outlinks:
            yield outlink, [pr / len(outlinks), needed_infos]
    
    def pr_reducer(self, key, values):
        global N, D
        prs = []
        outlinks = []
        for value in values:
            pr, infos = value
            prs.append(pr)
            if infos[0] == key:
                outlinks = infos[1]
        p = self.options.s
        text = str(key) + '\t' + str([p * sum(prs) + p * D/N + (1.0 - p)/N, outlinks]) + '\n'
        yield None, text
    
    def steps(self):
        return [
            MRStep(mapper=self.ip_mapper,
                   reducer=self.ip_reducer),
            MRStep(mapper=self.pr_mapper,
                   reducer=self.pr_reducer)
        ] * self.options.iter

if __name__ == '__main__':
    # Create the argument parser
    parser = argparse.ArgumentParser()

    # Add the arguments
    parser.add_argument('filename', type=str, help='the filename')
    parser.add_argument('--final-filename', type=str, default='pagerank', help='the number of iterations')
    parser.add_argument('--iter', type=int, default=5, help='the number of iterations')
    parser.add_argument('-s', type=float, default=0.85, help='the s value')
    parser.add_argument('--tollerance', type=float, default=0.00001, help='the tollerance value')

    # Parse the command line arguments
    args = parser.parse_args()

    # Retrieve the values
    filename = args.filename
    steps = args.iter
    s_value = args.s
    tollerance = args.tollerance
    final_filename = args.final_filename

    shutil.copyfile(filename, final_filename)

    prev_pr = {}
    new_pr = {}
    mr_job = MRPageRank(args=[final_filename, '--iter', str(steps), '-s', str(s_value)])

    change = 2
    iteration = 1
    while change > tollerance:
        with mr_job.make_runner() as runner:
            runner.run()
            change = 0

            with open(final_filename, 'w') as file:
                for _, line in mr_job.parse_output(runner.cat_output()):
                    node_id, info = line.split('\t')
                    pr, outlinks = eval(info)

                    if node_id in prev_pr:
                        change += abs(prev_pr[node_id] - pr)
                    elif iteration == 1:
                        change += abs(1/N - pr)

                    new_pr[node_id] = pr
                    file.write(line)

        prev_pr = new_pr
        print(f'Iteration: {iteration}')
        print(f'Change in l1 norm: {change}')
        iteration += steps

Overwriting mrpagerank.py


### Short Description of the Solution
The provided Python script, mrpagerank.py, implements the PageRank algorithm using the MRJob library for MapReduce simulation:

1. **Data Representation**: The script processes input data where each line represents a node (web page) in the graph. Each node is identified by a unique node ID and contains information about its PageRank value and a list of these outbound links.

2. **MapReduce Workflow**:

- Initialization Phase (ip_mapper and ip_reducer):
  - The ip_mapper function parses each line of input data and categorizes nodes into two groups:
    - Nodes with no outbound links (no outlinks).
    - Nodes with outbound links (outlinks).
  - The ip_reducer function receives these categorized nodes, calculates the total number of nodes (`N`), and determines the total dangling PageRank (`D`) from nodes with no outbound links.

- PageRank Calculation Phase (pr_mapper and pr_reducer):
  - The pr_mapper function takes processed nodes from the initialization phase and calculates the PageRank contribution for each outbound link of a node.
  - The pr_reducer function aggregates PageRank contributions received from different nodes and updates the PageRank value for each node using the damping factor (`s`).

3. **Iteration and Convergence**:

- The steps method defines the MapReduce job sequence to be executed multiple times (iter iterations) to approximate the PageRank values.
- The script runs in an iterative loop (while loop) where it:
  - Executes the defined MapReduce jobs.
  - Calculates the change in PageRank values between iterations to determine convergence (change).
  - Updates and writes the PageRank values to the output file (final_filename) after each iteration.

4. **Command Line Interface (CLI) Integration**:

- The script supports command-line arguments for specifying input file (filename), number of iterations (iter), damping factor (`s`), convergence tolerance (tolerance), and output file (final_filename).
- It uses these arguments to configure and execute the MRJob instance accordingly.


## Test on Florentine families

Represents the marriage and business connections among Florentine families during the Renaissance period.

Nodes: 16.

In my case I have decided to take only the marriage graph.

In [2]:
import xml.etree.ElementTree as ET

old_file = 'padgett.xml'
new_file = 'padgett_proc.txt'

# Parse the XML file
tree = ET.parse('padgett.xml')
root = tree.getroot()

# Extract the node IDs
node_ids = [node.attrib['id'] for node in root.findall('.//node')]
node_ids.sort()
id_to_num = {node_id: i for i, node_id in enumerate(node_ids)}
num_to_id = {i: node_id for i, node_id in enumerate(node_ids)}
n_nodes = len(node_ids)

with open(new_file, 'w') as f:
    # Extract the network information
    network = root.find('.//network[@id="PADGM"]')
    node_links = {num_node: [] for num_node in range(len(node_ids))}
    for link in network.findall('.//link'):
        if int(float(link.attrib['value'])) == 1:
            source = id_to_num[link.attrib['source']]
            target = id_to_num[link.attrib['target']]
            node_links[target].append(source)
    
    # Calculate the value of 1/N
    one_over_n = 1 / n_nodes

    for node_id, links in node_links.items():    
        # Write the formatted string to the file
        f.write(f'{node_id}\t[{one_over_n}, {links}]\n')

In [17]:
!python mrpagerank.py padgett_proc.txt --final-filename=padgett_pagerank.txt --iter=5 -s=0.85 --tollerance=0.00001 

No configs specified for inline runner
Iteration: 1
Change in l1 norm: 0.3750764118360122
No configs specified for inline runner
Iteration: 6
Change in l1 norm: 0.02852020773786826
No configs specified for inline runner
Iteration: 11
Change in l1 norm: 0.0065705498721270895
No configs specified for inline runner
Iteration: 16
Change in l1 norm: 0.001544554205905855
No configs specified for inline runner
Iteration: 21
Change in l1 norm: 0.0003961004680342467
No configs specified for inline runner
Iteration: 26
Change in l1 norm: 9.6453643254623e-05
No configs specified for inline runner
Iteration: 31
Change in l1 norm: 2.4765379896943274e-05
No configs specified for inline runner
Iteration: 36
Change in l1 norm: 6.164122293324248e-06


In [4]:
node_to_pr = {}
with open('padgett_pagerank.txt', 'r') as f:
    for line in f:
        node_id, node_info = line.split('\t')
        pr, outlinks = eval(node_info)
        node_id = int(node_id)
        node_to_pr[node_id] = pr

sorted_pr = sorted(node_to_pr.items(), key=lambda x: x[1], reverse=True)
sum = 0
for node_id, pr in sorted_pr:
    print(f'Node: {node_id} - {num_to_id[node_id]} - PageRank: {pr}')
    sum += pr

print()
print(f'Sum of PageRank: {sum}')

Node: 8 - MEDICI - PageRank: 0.1443732044757967
Node: 6 - GUADAGNI - PageRank: 0.09742342817750663
Node: 14 - STROZZI - PageRank: 0.08722615819345378
Node: 1 - ALBIZZI - PageRank: 0.07833903277824031
Node: 15 - TORNABUON - PageRank: 0.07057403646625554
Node: 12 - RIDOLFI - PageRank: 0.06888543350692938
Node: 4 - CASTELLAN - PageRank: 0.0686437086846023
Node: 3 - BISCHERI - PageRank: 0.06818004733016528
Node: 10 - PERUZZI - PageRank: 0.06720326913312744
Node: 13 - SALVIATI - PageRank: 0.06069640315889604
Node: 2 - BARBADORI - PageRank: 0.049803017635990134
Node: 9 - PAZZI - PageRank: 0.03569682980458627
Node: 5 - GINORI - PageRank: 0.03209693961276083
Node: 7 - LAMBERTES - PageRank: 0.030603551758192602
Node: 0 - ACCIAIUOL - PageRank: 0.030353949184486823
Node: 11 - PUCCI - PageRank: 0.009900990099009903

Sum of PageRank: 1.0


As we can see, the PageRank values have been calculated for each node in the Florentine families network based on the implemented MapReduce PageRank algorithm. These values represent the relative importance or centrality of each family within the network.

### Observations:

- **Medici (Node 8)** has the highest PageRank value, indicating its significant influence within the network.

- Other prominent families like **Guadagni (Node 6)**, **Strozzi (Node 14)**, and **Albizzi (Node 1)** also have relatively high PageRank values.

- Families with lower PageRank values, such as **Pazzi (Node 9)** and **Pucci (Node 11)**, are less central or influential in the network.

These PageRank results provide insights into the structure and importance of relationships among the Florentine families based on their historical connections.

## Wiki-Vote
Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt 

Wikipedia voting on promotion to administratorship (till January 2008). Directed edge A->B means user A voted on B becoming Wikipedia administrator.

Nodes: 7115 Edges: 103689

The PageRank algorithm will be applied to the Wiki-Vote dataset to analyze and rank the importance of users based on their voting behaviors. The goal is to identify the most influential users (nodes) within this network, where influence is determined by the number and quality of votes cast towards promoting other users.

In [7]:
id_to_outlinks = {}
with open('Wiki-Vote.txt', 'r') as f:
    for line in f:
        source, target = line.split()
        source = int(source)
        target = int(target)
        if source not in id_to_outlinks:
            id_to_outlinks[source] = []
        
        id_to_outlinks[source].append(target)

max_node = max(id_to_outlinks.keys())
for i in range(max_node):
    if i not in id_to_outlinks:
        id_to_outlinks[i] = []

n_nodes = len(id_to_outlinks)
one_over_n = 1 / n_nodes

with open('wiki_proc.txt', 'w') as f:
    for node_id, outlinks in id_to_outlinks.items():
        f.write(f'{node_id}\t[{one_over_n}, {outlinks}]\n')

In [22]:
!python mrpagerank.py wiki_proc.txt --final-filename=wiki_pagerank.txt --iter=3 -s=0.85 --tollerance=1e-7

No configs specified for inline runner
Iteration: 1
Change in l1 norm: 0.9165066958253977
No configs specified for inline runner
Iteration: 4
Change in l1 norm: 0.013735677346313988
No configs specified for inline runner
Iteration: 7
Change in l1 norm: 0.0009170520308486114
No configs specified for inline runner
Iteration: 10
Change in l1 norm: 0.00014227941286293005
No configs specified for inline runner
Iteration: 13
Change in l1 norm: 5.8420906086291134e-05
No configs specified for inline runner
Iteration: 16
Change in l1 norm: 3.3918083435145975e-05
No configs specified for inline runner
Iteration: 19
Change in l1 norm: 2.0823232477619004e-05
No configs specified for inline runner
Iteration: 22
Change in l1 norm: 1.2788067646022356e-05
No configs specified for inline runner
Iteration: 25
Change in l1 norm: 7.85347204011188e-06
No configs specified for inline runner
Iteration: 28
Change in l1 norm: 4.823013515728085e-06
No configs specified for inline runner
Iteration: 31
Change in 

Test the pagerank:

In [5]:
sum = 0
with open('wiki_pagerank.txt', 'r') as f:
    for line in f:
        node_id, node_info = line.split('\t')
        pr, outlinks = eval(node_info)
        sum += pr

print(f'Sum of PageRank: {sum}')

Sum of PageRank: 1.0000001553041662


## Web-Google
Directed graph (each unordered pair of nodes is saved once): web-Google.txt 

Webgraph from the Google programming contest, 2002

Nodes: 875713 Edges: 5105039

The PageRank algorithm will be applied to the Web-Google dataset to analyze and rank the importance of web pages based on their connectivity within the web graph. The goal is to identify the most influential web pages (nodes) that are likely to be important or central in terms of web navigation and information flow.

In [20]:
id_to_outlinks = {}
with open('web-Google.txt', 'r') as f:
    for line in f:
        source, target = line.split()
        source = int(source)
        target = int(target)
        if source not in id_to_outlinks:
            id_to_outlinks[source] = []
        
        id_to_outlinks[source].append(target)

max_node = max(id_to_outlinks.keys())
for i in range(max_node):
    if i not in id_to_outlinks:
        id_to_outlinks[i] = []

n_nodes = len(id_to_outlinks)
one_over_n = 1 / n_nodes

with open('google_proc.txt', 'w') as f:
    for node_id, outlinks in id_to_outlinks.items():
        f.write(f'{node_id}\t[{one_over_n}, {outlinks}]\n')

In [23]:
!python mrpagerank.py google_proc.txt --final-filename=google_pagerank.txt --iter=10 -s=0.85 --tollerance=1e-8

No configs specified for inline runner
Iteration: 1
Change in l1 norm: 0.9916116767044191
No configs specified for inline runner
Iteration: 11
Change in l1 norm: 0.02080242949370202
No configs specified for inline runner
Iteration: 21
Change in l1 norm: 0.0024964243999700923
No configs specified for inline runner
Iteration: 31
Change in l1 norm: 0.0003815088950087467
No configs specified for inline runner
Iteration: 41
Change in l1 norm: 6.366938252637513e-05
No configs specified for inline runner
Iteration: 51
Change in l1 norm: 1.1104888528229216e-05
No configs specified for inline runner
Iteration: 61
Change in l1 norm: 1.9913676943430594e-06
No configs specified for inline runner
Iteration: 71
Change in l1 norm: 3.622847804985308e-07
No configs specified for inline runner
Iteration: 81
Change in l1 norm: 6.655294526715902e-08
No configs specified for inline runner
Iteration: 91
Change in l1 norm: 1.2309390631369617e-08
No configs specified for inline runner
Iteration: 101
Change in

Test the pagerank:

In [6]:
sum = 0
with open('google_pagerank.txt', 'r') as f:
    for line in f:
        node_id, node_info = line.split('\t')
        pr, outlinks = eval(node_info)
        sum += pr

print(f'Sum of PageRank: {sum}')

Sum of PageRank: 0.9999999999967515


## Conclusions
The provided implementation demonstrates a simulation of the PageRank algorithm using the MapReduce approach with MRJob in Python.

It effectively breaks down the PageRank computation into map 
and reduce tasks, distributing the workload across multiple nodes.

The script supports iterative computation to approximate PageRank values and ensures convergence by monitoring changes in PageRank values between iterations.